In [ ]:
import os

import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

print(gpu_devices)

import warnings

warnings.filterwarnings('ignore', category=FutureWarning)
import warnings

warnings.filterwarnings("ignore")

import gc
import numpy as np
import keras
from keras.models import load_model
from tensorflow.keras.optimizers import Adam
import keras.backend as K
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from keras.layers import Activation, BatchNormalization, Conv1D, Dense, GlobalAveragePooling1D, Input, MaxPooling1D, \
    Lambda, Dropout
import random
import os
from sklearn import metrics
from scipy.stats import mode
from tqdm import tqdm
from scipy.spatial.distance import cosine
from multiprocessing import Pool
from resnet34 import *
from sklearn.model_selection import KFold
from keras import backend as K
import faiss
import datetime
from itertools import product
from annoy import AnnoyIndex
from keras import regularizers
from sklearn.metrics import *
from tensorflow_model_remediation import min_diff

seed_value = 1
# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(seed_value)
# 2. Set the `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)
# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)
# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)

In [ ]:
data_seed = 394
sub_fold_data_seed = 6

NOISE_RATIO = 0.0
chosen_k = 50
# large v1 31, mid v2 23, small v3 15
# stanford 50
FEATURE_LAYER_IDX = 50
FIND_k = False
WARM_UP_EPOCHS = 100
RETRAIN_EPOCHS = 200
WARMUP_FEATURE_SHAPE = 64  # THIS NEEDS TO BE CHANGED
K_RANGE = np.concatenate((np.arange(10, 50, 10), np.arange(50, 500, 50)))

In [ ]:
# @tf.custom_gradient
# def grad_reverse(x):
#     y = tf.identity(x)
#     def custom_grad(dy):
#         return -dy
#     return y, custom_grad

# class GradReverse(tf.keras.layers.Layer):
#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)

#     def call(self, x):
#         return grad_reverse(x)

In [ ]:
def qa_invariant_stanford_multitask(INPUT_LENGTH):
    inputs = Input(shape=[INPUT_LENGTH, 1])

    x = Conv1D(16,
               kernel_size=36,
               strides=4,
               padding='same',
               kernel_initializer='glorot_uniform',
               kernel_regularizer=regularizers.l2(l=0.0001)
               )(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling1D(pool_size=4, strides=None)(x)
    x = Dropout(0.4)(x)

    for i in range(3):
        x = stanford_identity_block(x, kernel_size=9, filters=32, stage=1, block=i)

    x = MaxPooling1D(pool_size=4, strides=None)(x)

    x = stanford_identity_block(x, kernel_size=3, filters=64, stage=2, block=i)
    x = stanford_identity_block(x, kernel_size=1, filters=64, stage=3, block=i)

    x = MaxPooling1D(pool_size=4, strides=None)(x)

    x = GlobalAveragePooling1D()(x)
 
    af_out = Dense(1, activation='sigmoid', name='af_output')(x)
    
    qa_x = GradReverse()(x)
    qa_out = Dense(1, activation='sigmoid', name='qa_output')(qa_x)

    m = Model(inputs, [af_out, qa_out], name='qa_invariant_stanford')
    return m


def qa_invariant_stanford(INPUT_LENGTH):
    inputs = Input(shape=[INPUT_LENGTH, 1])

    x = Conv1D(16,
               kernel_size=36,
               strides=4,
               padding='same',
               kernel_initializer='glorot_uniform',
               kernel_regularizer=regularizers.l2(l=0.0001)
               )(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling1D(pool_size=4, strides=None)(x)
    x = Dropout(0.4)(x)

    for i in range(3):
        x = stanford_identity_block(x, kernel_size=9, filters=32, stage=1, block=i)

    x = MaxPooling1D(pool_size=4, strides=None)(x)

    x = stanford_identity_block(x, kernel_size=3, filters=64, stage=2, block=i)
    x = stanford_identity_block(x, kernel_size=1, filters=64, stage=3, block=i)

    x = MaxPooling1D(pool_size=4, strides=None)(x)

    x = GlobalAveragePooling1D()(x)
 
    af_out = Dense(1, activation='sigmoid', name='af_output')(x)
    
    m = Model(inputs, af_out)
    return m

In [ ]:
print('Running', NOISE_RATIO, WARM_UP_EPOCHS)

X_train = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/X_train_all_{}.npy'.format(data_seed))
X_val = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/X_val_all_{}.npy'.format(data_seed))
X_test = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/X_test_all_{}.npy'.format(data_seed))

y_train_af = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/y_train_all_af_{}.npy'.format(data_seed))
y_val_af = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/y_val_all_af_{}.npy'.format(data_seed))
y_test_af = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/y_test_all_af_{}.npy'.format(data_seed))

y_train_qa = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/y_train_all_qa_{}.npy'.format(data_seed))
y_val_qa = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/y_val_all_qa_{}.npy'.format(data_seed))
y_test_qa = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/y_test_all_qa_{}.npy'.format(data_seed))

In [ ]:
y_train_qa[y_train_qa == 1] = 0
y_train_qa[y_train_qa == 2] = 1

In [ ]:
def flip_label(y, ratio):
    flip_marker = np.zeros((y.shape[0],))
    flip_idx = np.random.choice(range(y.shape[0]), int(y.shape[0]*ratio), replace=False)
    flip_marker[flip_idx] = 1
    flipped_y = y.copy()
    flipped_y[flip_idx] = 1 - flipped_y[flip_idx]
    return flipped_y, flip_marker

In [ ]:
noisy_y_train_af, flip_marker_train = flip_label(y_train_af, NOISE_RATIO)

In [ ]:
# def ReverseBinaryCrossEntropy(y_true, y_pred): 
#     y_true = tf.cast(y_true, tf.float32)
#     y_pred = tf.cast(y_pred, tf.float32)
    
#     y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
#     term_0 = (1 - y_true) * K.log(1 - y_pred + K.epsilon())  
#     term_1 = y_true * K.log(y_pred + K.epsilon())
    
#     return K.mean(term_0 + term_1, axis=0)

In [ ]:
def get_sample_weight(y):
    
    class_weight_af = compute_class_weight('balanced', np.unique(y), y)
    class_weight_af = {0: class_weight_af[0], 1: class_weight_af[1]}
    sample_weight_af = np.ones(shape=(len(y),))
    sample_weight_af[y == 1] *= class_weight_af[1]
    sample_weight_af[y == 0] *= class_weight_af[0]
    
    return sample_weight_af

In [ ]:
BATCH_SIZE = 128

af_train_main = tf.data.Dataset.from_tensor_slices((X_train, noisy_y_train_af, get_sample_weight(noisy_y_train_af))).batch(BATCH_SIZE)
af_train_main_qa_0 = tf.data.Dataset.from_tensor_slices((X_train[y_train_qa == 0], noisy_y_train_af[y_train_qa == 0], get_sample_weight(noisy_y_train_af[y_train_qa == 0]))).batch(BATCH_SIZE)
af_train_main_qa_1 = tf.data.Dataset.from_tensor_slices((X_train[y_train_qa == 1], noisy_y_train_af[y_train_qa == 1], get_sample_weight(noisy_y_train_af[y_train_qa == 1]))).batch(BATCH_SIZE)

af_val_main = tf.data.Dataset.from_tensor_slices((X_val, y_val_af)).batch(BATCH_SIZE)
af_val_main_qa_0 = tf.data.Dataset.from_tensor_slices((X_val[y_val_qa == 0], y_val_af[y_val_qa == 0])).batch(BATCH_SIZE)
af_val_main_qa_1 = tf.data.Dataset.from_tensor_slices((X_val[y_val_qa == 1], y_val_af[y_val_qa == 1])).batch(BATCH_SIZE)

train_dataset = min_diff.keras.utils.input_utils.pack_min_diff_data(af_train_main, af_train_main_qa_0, af_train_main_qa_1)
val_dataset = min_diff.keras.utils.input_utils.pack_min_diff_data(af_val_main, af_val_main_qa_0, af_val_main_qa_1)

In [ ]:
LR = 1e-3

min_diff_loss = min_diff.losses.MMDLoss()
min_diff_weight = 10.0

qa_invariant_model = qa_invariant_stanford(INPUT_LENGTH=X_train.shape[1])

min_diff_model = min_diff.keras.MinDiffModel(qa_invariant_model, min_diff_loss, min_diff_weight)

min_diff_model.compile(
    optimizer=Adam(lr=LR), loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.Precision(), keras.metrics.Recall()]
)

In [ ]:
qa_invariant_cp = 'qa_invariant_{}_{}_{}.h5'.format(WARM_UP_EPOCHS, NOISE_RATIO, data_seed)
qa_invariant_cp

hist = min_diff_model.fit(
    train_dataset,    
    callbacks=[
        ModelCheckpoint(filepath=qa_invariant_cp, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    ],
    
    validation_data=val_dataset,
    epochs=50,
    shuffle=True
)

In [ ]:
qa_invariant_model.load_weights(qa_invariant_cp)

In [ ]:
test_pred = qa_invariant_model.predict(X_test)
test_pred.shape

In [ ]:
af_test_pred = test_pred.squeeze()
af_test_pred_binary = af_test_pred.copy()
af_test_pred_binary[af_test_pred_binary > 0.5] = 1
af_test_pred_binary[af_test_pred_binary <= 0.5] = 0
af_test_f1 = f1_score(y_test_af, af_test_pred_binary)
af_test_f1

In [ ]:
# bce = tf.keras.losses.BinaryCrossentropy()
# a = tf.Variable(np.asarray([1, 0, 0, 1]), dtype=tf.float32)
# b = 1 - a
# c = a
# d = tf.Variable(np.asarray([0, 0, 0, 0]), dtype=tf.float32)
# print(bce(a, b))
# print(bce(a, c))
# print(bce(a, d))

# PacMap visualization warmup feature

In [ ]:
import pacmap
import matplotlib.pyplot as plt

In [ ]:
for idx, layer in enumerate(qa_invariant_model.layers):
    print(idx, layer.name)

In [ ]:
feat_extractor = Model(qa_invariant_model.input, qa_invariant_model.layers[FEATURE_LAYER_IDX].output)
feats = feat_extractor.predict(X_train)
feats = feats - np.mean(feats, axis=0)

In [ ]:
print(feats.shape)

In [ ]:
def build_annoy(warm_up_model_path, X_train, feature_layer_idx, save=True, ANNOY_NAME=None):
    best_warm_up = load_model(warm_up_model_path, custom_objects={'GradReverse': GradReverse})
    warm_up_feat_extractor = Model(best_warm_up.input, best_warm_up.layers[feature_layer_idx].output)
    warm_up_feats_raw = warm_up_feat_extractor.predict(X_train)
    warm_up_feats = warm_up_feats_raw - np.mean(warm_up_feats_raw, axis=0)
    f = warm_up_feats.shape[1]
    a = AnnoyIndex(f, 'angular')

    if ANNOY_NAME is None:
        ANNOY_NAME_ = 'stanford_all_{}_{}_{}_{}_mean_center.ann'.format(NOISE_RATIO, data_seed, 'qa_invariant', WARM_UP_EPOCHS)
    else:
        ANNOY_NAME_ = ANNOY_NAME

    if os.path.exists(ANNOY_NAME_):
        a.load(ANNOY_NAME_)
        print('Annoy loaded from', ANNOY_NAME_)
    else:
        print('Building AnnoyIndex for warm up feats of shape {}'.format(warm_up_feats.shape))
        a.set_seed(1)
        for idx, feat in enumerate(tqdm(warm_up_feats)):
            a.add_item(idx, feat)

        a.build(1000, n_jobs=15)  # 10 trees
        if save:
            a.save(ANNOY_NAME_)
            print('Annoy saved to', ANNOY_NAME_)
    del warm_up_feat_extractor, warm_up_feats
    gc.collect()

    return a

def build_annoy_with_model(warm_up_feat_extractor, X_train, feature_layer_idx, save=True, ANNOY_NAME=None):
    warm_up_feats_raw = warm_up_feat_extractor.predict(X_train)
    warm_up_feats = warm_up_feats_raw - np.mean(warm_up_feats_raw, axis=0)
    f = warm_up_feats.shape[1]
    a = AnnoyIndex(f, 'angular')

    if ANNOY_NAME is None:
        ANNOY_NAME_ = 'stanford_all_{}_{}_{}_{}_mean_center.ann'.format(NOISE_RATIO, data_seed, 'qa_invariant', WARM_UP_EPOCHS)
    else:
        ANNOY_NAME_ = ANNOY_NAME

    if os.path.exists(ANNOY_NAME_):
        a.load(ANNOY_NAME_)
        print('Annoy loaded from', ANNOY_NAME_)
    else:
        print('Building AnnoyIndex for warm up feats of shape {}'.format(warm_up_feats.shape))
        a.set_seed(1)
        for idx, feat in enumerate(tqdm(warm_up_feats)):
            a.add_item(idx, feat)

        a.build(1000, n_jobs=15)  # 10 trees
        if save:
            a.save(ANNOY_NAME_)
            print('Annoy saved to', ANNOY_NAME_)
    del warm_up_feat_extractor, warm_up_feats
    gc.collect()

    return a


annoy_index = build_annoy_with_model(feat_extractor, X_train, FEATURE_LAYER_IDX)

In [ ]:
MODEL_NAME = 'qa_invariant'

In [ ]:
import multiprocessing.pool as mpp

def istarmap(self, func, iterable, chunksize=1):
    """starmap-version of imap
    """
    if self._state != mpp.RUN:WARM_UP_EPOCHS 
        raise ValueError("Pool not running")

    if chunksize < 1:
        raise ValueError(
            "Chunksize must be 1+, not {0:n}".format(
                chunksize))

    task_batches = mpp.Pool._get_tasks(func, iterable, chunksize)
    result = mpp.IMapIterator(self._cache)
    self._taskqueue.put(
        (
            self._guarded_task_generation(result._job,
                                          mpp.starmapstar,
                                          task_batches),
            result._set_length
        ))
    return (item for chunk in result for item in chunk)

def annoy_pacmap_prepare_child(idx_t, annoy_index_path, f):

    annoy_index = AnnoyIndex(f, 'angular')
    annoy_index.load(annoy_index_path)
    return idx_t, annoy_index.get_nns_by_item(idx_t, 1000 + 1)


def annoy_pacmap_prepare(feats, annoy_index):
    
    tmp_annoy_index_path = 'tmp_annoy_index_{}_{}.ann'.format(NOISE_RATIO, data_seed)
    f = WARMUP_FEATURE_SHAPE
    annoy_index.save(tmp_annoy_index_path)

    pool_args = []
    for idx_t in range(len(X_train)):
        pool_args.append([idx_t, tmp_annoy_index_path, f])
    
    n, dim = feats.shape
    nbrs = np.zeros((n, 1000), dtype=np.int32)
    
    mpp.Pool.istarmap = istarmap
    pool = Pool(processes=15)

    for rst in tqdm(pool.istarmap(annoy_pacmap_prepare_child, pool_args), total=len(pool_args)):
        nbrs[rst[0], :] = rst[1][1:]

    pool.terminate()
    
    return nbrs



nbrs = annoy_pacmap_prepare(feats, annoy_index)

In [ ]:
np.save('pacmap_nbrs_{}_{}_{}_{}_mean_center.npy'.format(1000, WARM_UP_EPOCHS, NOISE_RATIO, data_seed), nbrs)

In [ ]:
n_neighbors = chosen_k
n, dim = feats.shape

scaled_dist = np.ones((n, n_neighbors)) # No scaling is needed
scaled_dist = scaled_dist.astype(np.float32)

pair_neighbors = pacmap.sample_neighbors_pair(feats.astype(np.float32), scaled_dist, nbrs, np.int32(n_neighbors))

embedding = pacmap.PaCMAP(n_dims=2, n_neighbors=n_neighbors, MN_ratio=0.5, FP_ratio=2.0, pair_neighbors=pair_neighbors)
X_transformed = embedding.fit_transform(feats, init="pca")
X_transformed.shape

In [ ]:
np.save('pacmap_transformed_{}_{}_{}_{}_mean_center.npy'.format(1000, WARM_UP_EPOCHS, NOISE_RATIO, data_seed), X_transformed)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(X_transformed[:, 0][y_train_af==0], X_transformed[:, 1][y_train_af==0], label='Clean Non-AF', s=1, alpha=0.5, c='tab:blue')
plt.scatter(X_transformed[:, 0][y_train_af==1], X_transformed[:, 1][y_train_af==1], label='Clean AF', s=1, alpha=0.5, c='tab:red')
plt.legend(markerscale=10)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(X_transformed[:, 0][noisy_y_train_af==0], X_transformed[:, 1][noisy_y_train_af==0], label='Noisy Non-AF', s=1, alpha=0.1, c='tab:blue')
plt.scatter(X_transformed[:, 0][noisy_y_train_af==1], X_transformed[:, 1][noisy_y_train_af==1], label='Noisy AF', s=1, alpha=0.1, c='tab:red')
plt.legend(markerscale=10)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
# plt.scatter(X_transformed[:, 0][y_train_qa==0], X_transformed[:, 1][y_train_qa==0], label='Noisy Non-AF', s=1, alpha=0.5, c='tab:blue')
plt.scatter(X_transformed[:, 0][y_train_qa==1], X_transformed[:, 1][y_train_qa==1], label='Noisy AF', s=1, alpha=0.5, c='tab:red')
plt.legend(markerscale=10)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(X_transformed[:, 0][y_train_qa==0], X_transformed[:, 1][y_train_qa==0], label='Noisy Non-AF', s=1, alpha=0.5, c='tab:blue')
# plt.scatter(X_transformed[:, 0][y_train_qa==1], X_transformed[:, 1][y_train_qa==1], label='Noisy AF', s=1, alpha=0.5, c='tab:red')
plt.legend(markerscale=10)
plt.show()

In [ ]:
y_train_qa = np.load('/home/chengstark/Dev/ssd/robust_learning/stanford_reboot/cheecky_reworked_data/y_train_all_qa_{}.npy'.format(data_seed))

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(X_transformed[:, 0][y_train_qa==1], X_transformed[:, 1][y_train_qa==1], label='Acceptable', s=1, alpha=0.1, c='tab:orange')
plt.scatter(X_transformed[:, 0][y_train_qa==2], X_transformed[:, 1][y_train_qa==2], label='Good', s=1, alpha=0.1, c='tab:blue')
plt.scatter(X_transformed[:, 0][y_train_qa==0], X_transformed[:, 1][y_train_qa==0], label='Bad', s=1, alpha=0.1, c='tab:red')

plt.legend(markerscale=10)
plt.show()

print(np.unique(y_train_qa, return_counts=True))

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(X_transformed[:, 0][y_train_qa==1], X_transformed[:, 1][y_train_qa==1], label='Acceptable', s=1, alpha=0.1, c='tab:orange')
# plt.scatter(X_transformed[:, 0][y_train_qa==2], X_transformed[:, 1][y_train_qa==2], label='Good', s=1, alpha=0.1, c='tab:blue')
plt.scatter(X_transformed[:, 0][y_train_qa==0], X_transformed[:, 1][y_train_qa==0], label='Bad', s=1, alpha=0.1, c='tab:red')

plt.legend(markerscale=10)
plt.show()

print(np.unique(y_train_qa, return_counts=True))

In [ ]:
fig = plt.figure(figsize=(10, 10))
# plt.scatter(X_transformed[:, 0][y_train_qa==1], X_transformed[:, 1][y_train_qa==1], label='Acceptable', s=1, alpha=0.1, c='tab:orange')
plt.scatter(X_transformed[:, 0][y_train_qa==2], X_transformed[:, 1][y_train_qa==2], label='Good', s=1, alpha=0.1, c='tab:blue')
# plt.scatter(X_transformed[:, 0][y_train_qa==0], X_transformed[:, 1][y_train_qa==0], label='Bad', s=1, alpha=0.1, c='tab:red')

plt.legend(markerscale=10)
plt.show()

print(np.unique(y_train_qa, return_counts=True))